## **1. Install Dependencies**

In [4]:
# ─── Install Dependencies ─────────────────────────────────────────────────────
!apt-get update -y && apt-get install -y tesseract-ocr
!pip install --quiet \
    gradio pytesseract Pillow requests \
    sentence-transformers faiss-cpu transformers \
    torch torchvision torchaudio

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

## **2. Importing Required Libraries**

In [15]:
# Importing regular expressions, HTTP request, and Gradio for the interface
import re
import requests
import gradio as gr

# To handle image data in memory
from io import BytesIO

# For image processing
from PIL import Image

# For Optical Character Recognition (OCR)
import pytesseract

# Sentence embedding model for semantic search
from sentence_transformers import SentenceTransformer

# FAISS for efficient vector similarity search
import faiss

# Transformers for text summarization / generation tasks
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


## **3. Menu Image URLs for Each Restaurant**

In [16]:
# ─── Dictionary to Store Static Menu Image URLs ──────────────────────────────
# Each restaurant is a key in the dictionary, mapped to a list of image URLs of their menu
RESTAURANT_MENUS = {
   "Pukhtaan": [
        "https://b.zmtcdn.com/data/menus/853/21580853/feb85f6fa20259fe481a7bc440c24476.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/10691ec61acb78405f5961a3d22a42f2.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/460c2011d7dfda6acf1be4dbf8a73da7.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/7e77459a125abd77aed34f7a882ad7e7.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/e08b90bf17c69fbe518c79c11ff6db7f.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/7672cc353331bd91ed471104de6192b5.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/d0f40d75623a98ebc9e8527c2bc0d5e6.jpg",
        "https://b.zmtcdn.com/data/menus/853/21580853/ead678286bc10afb1f1a6c3bb94ecf10.jpg",
    ],
    "Connaught_Club_House": [
        "https://b.zmtcdn.com/data/menus/106/19295106/faf36abd62cb22e25492a1e51d36d971.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/6d533623bfb316c927d5087a1be26f1b.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/92fdde9055013c6ab9ba7e9b76c1770d.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/d59c8c08538a220438fab471790d2b3b.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/d5ff7bf34e11499d79ee59b5492e892d.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/5757370e09010293de96576c65c499f3.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/e784555ca0fac7f4b2c78ad12d708108.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/cf56b2e45236ab81e3adeebde09937e5.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/b3a95e356d17115f48f31e488fa14510.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/db6351e453b6a553e2a95a029ccd5c93.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/63668b33e2cd94b084aba2400a32302d.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/28c47c6101f4b6525ae0f00054685f7b.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/8580a3bc941162b7ac0fcf6a2b29408f.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/9e199b47a96c620137325a2edaa257ea.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/a48b905eda7e351ec9d16be38263c6ce.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/e1256a12cfc8d2671887daf5849462c3.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/407cf6d0c92226a1e8d3c2e306bc7eee.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/a06bc4526e30e16cd66e7fc33680ddd1.jpg",
        "https://b.zmtcdn.com/data/menus/106/19295106/f43d42f35215a54255c2b3c5e7bc361d.jpg"
    ],
    "Local": [
        "https://b.zmtcdn.com/data/menus/360/18382360/4a261b83c8d83e45c90ba18738606383.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/e594cde054e3b9cd74bada9713d45647.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/09f07621c6c933873eaa0e3cc2d8cee6.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/9df0e7ea170f93df343a776cbb433b2e.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/c7a32dfe8174158daf3395a2d8a3122a.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/6213b5c0e0e6304b496aa40ec29ee7c0.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/62bb0f0ab165aed9aac2f07386be6ce0.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/9bb62a9813fd95783befe1f938dff819.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/39c8d9378a98634edad40bd2e21c8269.jpg",
        "https://b.zmtcdn.com/data/menus/360/18382360/9059e84501429b654a3b187f8fa05f3b.jpg"
    ],
    "Roofberries": [
        "https://b.zmtcdn.com/data/menus/098/20020098/2bee857f808bf90d3a339432e3a9a97a.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/177310b350732ea411fce4ee922ce00c.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/c96d1f09849034960eb591d821a9255e.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/8eba87e7fc2ba0a32cb8ca73ce72e159.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/331391a9f9e30a09b1fe30a87c89b088.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/4fbe208100d596c5239518143fef18be.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/e232cbceaa3ac5ac4d66f91a18156cb3.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/e232cbceaa3ac5ac4d66f91a18156cb3.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/9d27ecac94a5815f62fe77fed142996b.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/40787d97f1db990c4199bda163a3f006.jpg",
        "https://b.zmtcdn.com/data/menus/098/20020098/ce5f056e39226e5d1cc705b197116fc0.jpg"
    ],
    "QEY": [
        "https://b.zmtcdn.com/data/menus/303/21180303/5fee9979bef7ea53f215f1eff035a98f.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/4ab2a194e0ef6b0976e94d55401aa815.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/cf7fb24e217cf17bf8884e3415bb333c.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/81cdfc3844a6c8adf4df6703f58435cc.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/a1843c6cc22ea5a9a8e00ae79e75fdf2.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/8466ea20afd3ba86a4742cd77b438def.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/8466ea20afd3ba86a4742cd77b438def.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/400e7dbc0b344795a3e937d0f3b7e07e.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/035bdb561621fc8ab19a89561e13f977.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/035bdb561621fc8ab19a89561e13f977.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/6f9151dff3d95c75f045151497b38fe9.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/48ba28da967d96653d8255d4197988c1.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/60b2ac9ef657dde53df1bf3026ab7083.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/73b22be7bae1ac5650e1ee9084a186ac.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/facad6ef851470f4313c134f2e59265e.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/8afaa475084fc676f842ec067f4888fb.jpg",
        "https://b.zmtcdn.com/data/menus/303/21180303/fde5ab708e230a214ea2b0defa1f3f1a.jpg"
    ],
}

## **4. OCR-Based Menu Digitization & Text Cleaning**

In [7]:
# Function: ocr_text(url)
# Purpose: Takes an image URL, downloads the image, and extracts text using Tesseract OCR
# Parameters:
#   url (str): The URL of the image to process
# Returns:
#   str: Raw extracted text from the image
def ocr_text(url):
    r = requests.get(url, timeout=15)  # Fetch image with timeout
    r.raise_for_status()  # Raise error if request fails
    img = Image.open(BytesIO(r.content))  # Load image from byte stream
    return pytesseract.image_to_string(img)  # Perform OCR and return text


# Function: clean_txt(txt)
# Purpose: Cleans and standardizes the raw OCR text for improved processing
# Parameters:
#   txt (str): The raw text extracted from an image
# Returns:
#   str: Cleaned and corrected version of the text
def clean_txt(txt):
    # Remove unwanted special characters (retain prices, symbols, and common punctuations)
    txt = re.sub(r'[^A-Za-z0-9\s\.,₹$€¥¢&+()/-]', '', txt)

    # Dictionary of common OCR misreads and domain-specific corrections
    fixes = {
        r'\b(O|0)e\b':         'Of',                      # Correct misread 'Of'
        r'\bPaneera?\b':       'Paneer',                  # Correct OCR spelling of Paneer
        r'\bMurgh\b':          'Chicken',                 # Standardize 'Murgh' to 'Chicken'
        r'\bCCH\b':            'Connaught_Club_House',    # Expand restaurant abbreviation
        r'\bR\s*(\d+)':        r'₹\1',                    # Normalize currency notation
        r'\b(\d{1,3})\s*-\s*(\d{1,3})\b': r'₹\1-₹\2',      # Normalize price ranges
        r'\b[Tt]andoori\s+Alo\b':'Tandoori Aloo'          # Correct common dish OCR error
    }

    # Apply all replacements using regex
    for pat, rep in fixes.items():
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)

    return txt


## **5. Parsing Menu Text and Tagging Items**

In [17]:
# ─── Step 5.1: Parse Cleaned Menu Text into Structured Items ─────────────────────
def parse_menu(raw):
    # Clean and split input into non-empty lines
    lines = [l.strip() for l in clean_txt(raw).splitlines() if l.strip()]

    # Regular expression to capture price patterns like ₹120, Rs. 250, etc.
    price_re = re.compile(r'(₹|Rs\.?|INR)\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', re.I)

    items, cur = [], None  # List to store parsed items, and a placeholder for current item

    for ln in lines:
        m = price_re.search(ln)

        # If line contains a price and we have a current item, assign the price and store the item
        if m and cur:
            cur['price'] = m.group().strip()
            items.append(cur)
            cur = None

        # If line looks like a new item name (starts with uppercase and is sufficiently long)
        elif ln[0].isupper() and len(ln) > 3:
            if cur: items.append(cur)  # Save previous item if still pending
            cur = {'name': ln, 'price': '', 'description': ''}  # Start new item

        # Otherwise, treat the line as a description for the current item
        elif cur:
            cur['description'] += ' ' + ln

    if cur: items.append(cur)  # Add the last item if not already added
    return items

In [18]:
# ─── Step 5.2: Tag Items with Dietary and Spice Information ──────────────────────
def tag_items(items):
    # Non-vegetarian keywords to flag non-veg dishes
    diet_non = {'chicken','mutton','fish','prawn','meat','lamb','beef','egg','shrimp','crab'}

    # Grouped keywords to identify spice levels
    spice_3  = {'extra spicy','fiery','blazing','scorching','ghost pepper'}
    spice_2  = {'spicy','chilli','hot','pepper','masala','jalapeno','szechuan','cayenne'}
    spice_1  = {'mild','paprika','tandoor','tikka','curry'}

    for it in items:
        text = (it['name'] + ' ' + it['description']).lower()

        # Determine if item is vegetarian by checking absence of non-veg keywords
        it['is_veg'] = not any(w in text for w in diet_non)

        # Determine spice level based on keyword matches
        lvl = 0
        if any(w in text for w in spice_1): lvl = 1
        if any(w in text for w in spice_2): lvl = 2
        if any(w in text for w in spice_3): lvl = 3
        it['spice_level'] = lvl

    return items


## **6. Build Restaurant Menu Dataset**

In [20]:
restaurant_data = {}  # Dictionary to store structured data for each restaurant

# Loop through each restaurant and its corresponding list of menu image URLs
for r, urls in RESTAURANT_MENUS.items():
    # Apply OCR to all menu images and join the text outputs
    pages = [ocr_text(u) for u in urls]

    # Parse the joined text into structured items (name, description, price)
    items = parse_menu("\n\n".join(pages))

    # Tag items with dietary info (veg/non-veg) and spice levels
    tagged_items = tag_items(items)

    # Store the tagged menu items in the dictionary under the restaurant name
    restaurant_data[r] = {'items': tagged_items}


## **7. Generate Embeddings & Build FAISS Index**

In [21]:
docs, meta = [], []  # `docs` holds text for embedding, `meta` stores related info

# Prepare document strings and metadata for each menu item across restaurants
for r, info in restaurant_data.items():
    for it in info['items']:
        # Create searchable string combining name, description, and restaurant name
        docs.append(f"{it['name']} — {it['description']} [{r}]")

        # Store metadata: restaurant name + item details (price, tags, etc.)
        meta.append({'restaurant': r, **it})

# Load a lightweight semantic embedder model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Convert text documents into vector embeddings
embs = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)

# Initialize a FAISS index and add all embeddings for fast similarity search
index = faiss.IndexFlatL2(embs.shape[1])
index.add(embs)


Batches:   0%|          | 0/42 [00:00<?, ?it/s]

## **8. Setup RAG (Retrieval-Augmented Generation) with FLAN-T5**

In [22]:
# Load pretrained FLAN-T5 model and tokenizer for generating text-based answers
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model     = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Create a text2text generation pipeline using the model
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)

# Define RAG-style question-answering function
def rag_answer(q, k=5):
    # Convert the question to an embedding vector
    qe = embedder.encode([q], convert_to_numpy=True)

    # Search for top-k most relevant menu items using FAISS
    _, I = index.search(qe, k)

    # Construct a context string from the retrieved menu items
    ctx = "\n".join(f"- {docs[i]}" for i in I[0])

    # Build a prompt combining context and the question
    prompt = f"You are a restaurant menu expert.\nContext:\n{ctx}\n\nQuestion: {q}\nAnswer:"

    # Generate the answer using the FLAN-T5 pipeline
    return pipe(prompt, do_sample=False)[0]['generated_text'].strip()


Device set to use cpu


## **9. Format Menu Output for Display**

In [23]:
def format_menu_output(filtered):
    """
    Nicely formats filtered menu items for clean and readable display.

    Parameters:
        filtered (list): A list of menu items, each tagged with metadata like
                         restaurant name, veg/non-veg, spice level, and price.

    Returns:
        str: Formatted string with icons and groupings by restaurant.
    """
    output = ""
    current_restaurant = None

    # Iterate through the list of filtered menu items
    for item in filtered:
        # Add a new restaurant section header if restaurant changes
        if item['restaurant'] != current_restaurant:
            current_restaurant = item['restaurant']
            output += f"\n---\n### 🍽 {current_restaurant.replace('_',' ')}\n"

        # Add veg/non-veg icon
        veg_icon = "🌱 Veg" if item['is_veg'] else "🍖 Non-Veg"

        # Show spice level with 🔥 icons or 🌿 for mild
        spice_icon = "🔥" * item['spice_level'] or "🌿 Mild"

        # Fallback if price is not available
        price = item['price'] or "Price N/A"

        # Append item info to the output string
        output += (
            f"\n*{item['name']}*  \n"
            f"{veg_icon} | {spice_icon}  \n"
            f"💰 {price}  \n"
            f"{item['description'].strip()}\n"
        )

    return output.strip()


## **10. Build the Chat Function**

In [24]:
# ─── Step 8: Chat Function ─────────────────────────────────────────────────────

def chat_fn(query):
    """
    Main query interface for users to interact with the restaurant menu chatbot.
    It performs semantic search, applies keyword filtering, and optionally
    uses RAG (Retrieval-Augmented Generation) to respond.

    Parameters:
        query (str): The user's natural language query.

    Returns:
        str: A formatted response either from matched menu items or generated text.
    """
    ql = query.strip().lower()

    # 8.1 Semantic retrieval using sentence embeddings
    qe = embedder.encode([query], convert_to_numpy=True)
    _, I = index.search(qe, 5)
    hits = [meta[i] for i in I[0]]

    # 8.2 Keyword filtering for more direct, accurate matches
    terms = set(ql.split())
    filtered = [h for h in hits if terms & set((h['name'] + ' ' + h['description']).lower().split())]

    # Return formatted menu if there are direct matches
    if filtered:
        return format_menu_output(filtered)

    # 8.3 Fallback: Use RAG pipeline for broader questions (e.g., comparisons, recommendations)
    return rag_answer(query)


Launch Gradio UI

In [25]:
import gradio as gr

# Creating the Gradio interface for the restaurant menu chatbot.
gr.Interface(
    fn=chat_fn,  # The function to be invoked on user input (chat_fn from previous steps)
    inputs=gr.Textbox(  # Input field where users can type queries
        lines=2,  # Number of lines for the input box
        placeholder="e.g. show me chicken dishes"  # Placeholder text for guidance
    ),
    outputs="markdown",  # Output format for displaying the results
    title="🍴 Restaurant Menu Explorer",  # Title of the interface
    description="Ask about dishes, dietary filters, spice, prices, comparisons, or any free‐text query."  # Interface description
).launch(share=True)  # Launches the interface and shares a public link for access


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbb5eaa9649cf61a40.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
